In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import (
    MobileNetV2,
    ResNet50,
    EfficientNetB0
)

import os

# ================= PARAMETERS =================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 3

TRAIN_DIR = "dataset/train/train"
VAL_DIR = "dataset/validation/validation"
TEST_DIR = "dataset/test/test"

# ================= DATA GENERATORS =================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# ================= MODELS TO TEST =================
models_to_try = {
    "MobileNetV2": MobileNetV2,
    "EfficientNetB4": EfficientNetB4,
    "DenseNet121": DenseNet121,
    "ResNet50": ResNet50
}

best_accuracy = 0
best_model_name = ""
best_model = None

# ================= TRAINING LOOP =================
for model_name, model_class in models_to_try.items():
    print(f"\n🚀 Training {model_name}...\n")

    base_model = model_class(
        weights="imagenet",
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(NUM_CLASSES, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        train_data,
        epochs=EPOCHS,
        validation_data=val_data,
        verbose=1
    )

    loss, accuracy = model.evaluate(test_data, verbose=0)
    print(f"✅ {model_name} Test Accuracy: {accuracy * 100:.2f}%")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name
        best_model = model

# ================= SAVE BEST MODEL =================
best_model.save("best_plant_disease_model.h5")

print("\n================ FINAL RESULT ================")
print(f"🏆 Best Model      : {best_model_name}")
print(f"🎯 Best Accuracy   : {best_accuracy * 100:.2f}%")
print("💾 Model Saved As  : best_plant_disease_model.h5")
print("=============================================")


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 151 images belonging to 3 classes.

🚀 Training MobileNetV2...

Epoch 1/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.5303 - loss: 1.0496 - val_accuracy: 0.8833 - val_loss: 0.4961
Epoch 2/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.7648 - loss: 0.5831 - val_accuracy: 0.9167 - val_loss: 0.3216
Epoch 3/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.8480 - loss: 0.3939 - val_accuracy: 0.9333 - val_loss: 0.2499
Epoch 4/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.8903 - loss: 0.3234 - val_accuracy: 0.9333 - val_loss: 0.2200
Epoch 5/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 116s 3s/step - accuracy: 0.9085 - loss: 0.2708 - val_accuracy: 0.9333 - val_loss: 0.1964
Epoch 6/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.9221 - loss: 0.2324 - val_accuracy: 0.9333 - val_loss: 0.1848
Epoch 7/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9289 - loss: 0.2143 - val_a

✅ EfficientNetB0 Test Accuracy: 33.77%

================ FINAL RESULT ================
🏆 Best Model      : MobileNetV2
🎯 Best Accuracy   : 97.35%
💾 Model Saved As  : best_plant_disease_model.h5
